<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/BoostedBinaryClassifers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls
%cd drive/My Drive/ConfidentMachineTranslation/flores
!ls
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive  sample_data
/content/drive/My Drive/ConfidentMachineTranslation/flores
'=0.5.0'			 Ensembling
 analysis			 FLORES.ipynb
 Analysis			 LanguageAnalysis.ipynb
 backward_models		 language_models
 BoostedBinaryClassifers.ipynb	 LM_Thresholding.ipynb
 checkpoints			 NCD_Analysis.ipynb
 ClassificationDataset		 NNClassification.ipynb
 configs			 noisychannel
 data				 NoisyChannel.ipynb
 data-bin			 Resources
 Ensembles			 scripts
/content/drive/My Drive/Confid

In [2]:
# prints how much GPU RAM is available
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=129d324cd80b9274738ee449d4d135d4d5055be2ceee15df0d2d04583ce244df
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 160.1 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [5]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import numpy as np
from itertools import zip_longest

def printDatasetClassProp(Y): 
    classes = {}
    total = len(Y)
    for i in Y:
        if i in classes:
            classes[i] += 1
        else:
            classes[i] = 1
    
    for cls in classes:
        print("Proportion in class " + str(cls) + " = " + str(classes[cls]/total))

def datasetReader(featureFile, labelFile):
    files = [featureFile, labelFile]

    X = []
    Y = []

    for lines in zip_longest(*files, fillvalue=''):
        currX, currY = lines[0], float(lines[1].strip("\n"))
        Xarr = []
        features = currX.split()
        for feature in features:
            Xarr.append(float(feature.strip(",").strip("\n")))
        X.append(Xarr)
        Y.append(currY)
    
    return np.array(X), np.array(Y)


def trainRandomForestClassifier(X, Y, maxDepth=8):
    print("Training Random Forest classifier")
    clf = RandomForestClassifier(max_depth=maxDepth, random_state=42)
    clf.fit(X, Y)
    return clf

def trainAdaBoostClassifier(X, Y, estimators=100):
    print("Training AdaBoosted Decision Tree classifier")
    clf = AdaBoostClassifier(n_estimators=estimators, random_state=42)
    clf.fit(X, Y)
    return clf

def trainGradientBoostingClassifier(X, Y):
    print("Training Graident Boosted classifier")
    clf = GradientBoostingClassifier(random_state=42)
    clf.fit(X, Y)
    return clf

def calculateAccuracy(predictedClasses, groundTruth):
    correct_accepted = 0
    total_accepted = 0

    correct_rejected = 0
    total_rejected = 0

    for i in range(len(predictedClasses)):
        if groundTruth[i] == 1:
            total_accepted += 1
            if predictedClasses[i] == groundTruth[i]:
                correct_accepted += 1
        else:
            total_rejected += 1
            if predictedClasses[i] == groundTruth[i]:
                correct_rejected += 1


    print("Correctly accepted = " + str(correct_accepted/total_accepted))
    print("Incorrectly accepted = " + str(1 - correct_rejected/total_rejected))
    print("Correctly rejected = " + str(correct_rejected/total_rejected))
    print("Incorrectly rejected = " + str(1 - correct_accepted/total_accepted))

    print("Total Accuracy = " + str((correct_accepted + correct_rejected)/(total_accepted + total_rejected)))
trainset = "valid"
testset = "test"

trainFeatures = open("ClassificationDataset/"+trainset+"/features.txt")
testFeatures = open("ClassificationDataset/"+testset+"/features.txt")

trainLabels = open("ClassificationDataset/"+trainset+"/result.txt")
testLabels = open("ClassificationDataset/"+testset+"/result.txt")

trainX, trainY = datasetReader(trainFeatures, trainLabels)
testX, testY = datasetReader(testFeatures, testLabels)

printDatasetClassProp(trainY)


classifiers = [trainRandomForestClassifier, trainAdaBoostClassifier, trainGradientBoostingClassifier]

for classifier in classifiers:
    print("#################################################")
    curr = classifier(trainX, trainY)
    predictions = np.array(curr.predict(testX))
    calculateAccuracy(predictions, testY)
    print("#################################################")



Proportion in class 0.0 = 0.8014849550605705
Proportion in class 1.0 = 0.19851504493942945
#################################################
Training Random Forest classifier
Correctly accepted = 0.24200278164116829
Incorrectly accepted = 0.03308128544423439
Correctly rejected = 0.9669187145557656
Incorrectly rejected = 0.7579972183588317
Total Accuracy = 0.783068783068783
#################################################
#################################################
Training AdaBoosted Decision Tree classifier
Correctly accepted = 0.29763560500695413
Incorrectly accepted = 0.060491493383742934
Correctly rejected = 0.9395085066162571
Incorrectly rejected = 0.7023643949930458
Total Accuracy = 0.7767195767195767
#################################################
#################################################
Training Graident Boosted classifier
Correctly accepted = 0.26286509040333794
Incorrectly accepted = 0.047258979206049156
Correctly rejected = 0.9527410207939508
Incorrectly re